In [1]:
import os 

In [2]:
os.chdir('/home/yehudan/Desktop/DecisionTree/DecisionTree')

In [1]:
import time
from datasets import mse_comparison 
from scipy.io.arff import loadarff
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('datasets/Pakistan_Largest_Ecommerce_Dataset.csv')

/data/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.head()

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,...,Month,Customer Since,M-Y,FY,Customer ID,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1950.0,1.0,1950.0,100147443,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,1.0,NaN,NaN,NaN,NaN,NaN
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,...,7.0,2016-7,7-2016,FY17,2.0,NaN,NaN,NaN,NaN,NaN
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,3.0,NaN,NaN,NaN,NaN,NaN
3,211135.0,complete,7/1/2016,kcc_krone deal,360.0,1.0,60.0,100147446,Beauty & Grooming,R-FSD-52352,...,7.0,2016-7,7-2016,FY17,4.0,NaN,NaN,NaN,NaN,NaN
4,211136.0,order_refunded,7/1/2016,BK7010400AG,555.0,2.0,1110.0,100147447,Soghaat,\N,...,7.0,2016-7,7-2016,FY17,5.0,NaN,NaN,NaN,NaN,NaN


In [6]:
def move_columns_to_end(df,col1,col2=None):
    cols = list(df.columns.values) #Make a list of all of the columns in the df
    cols.pop(cols.index(col1)) #Remove col1 from list
    if col2:
        cols.pop(cols.index(col2)) #Remove col2 from list
    if col2:
        df = df[cols+[col1,col2]] #Create new dataframe with columns in the order you want
    else:
        df = df[cols+[col1]]
    return df

In [7]:
pakistan_columns = ['grand_total', 'category_name_1', 'payment_method', 'Customer ID']
pakistan_filtered = df[pakistan_columns]
grouped_data = pakistan_filtered.groupby(by=['category_name_1', 'Customer ID'], as_index=False).agg({'grand_total': 'sum'})
pivoted = grouped_data.pivot(index='Customer ID', columns='category_name_1', values='grand_total')
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

In [8]:
pivoted

,Customer ID,Appliances,Beauty & Grooming,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Mobiles & Tablets,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950.0,NaN
1,2.0,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270.0,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2490.00,2490.00,NaN,NaN,NaN,NaN,2450.0,NaN
3,4.0,285592.7,34219.3,NaN,99.0,593501.2,999.0,22185.0,2250.0,97804.55,1257478.35,1249.0,495.0,32990.0,94530.0,29158.5,115044.85
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1110.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115306,115322.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2596.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115307,115323.0,44499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115308,115324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,549.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115309,115325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1199.0,NaN


In [9]:
pivoted[pivoted.notnull().sum(axis=1)>=9].shape

(1325, 17)

In [10]:
final_df = pivoted[pivoted.notnull().sum(axis=1)>=9]
final_df = final_df.drop(['Customer ID'], axis='columns')
final_df = final_df.fillna(0)
final_df.shape

(1325, 16)

In [11]:
pivoted.corr()

,Customer ID,Appliances,Beauty & Grooming,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Mobiles & Tablets,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N
Customer ID,1.000000,0.003700,-0.016565,-0.030022,0.061213,0.023333,0.006154,0.009600,0.026787,-0.021875,0.037296,0.041667,-0.031247,0.023158,0.003918,-0.013441,-0.113714
Appliances,0.003700,1.000000,0.193731,0.043866,0.263430,0.479368,0.163356,0.223021,0.100470,0.255954,0.238027,0.150902,0.016345,0.184994,0.116739,0.195529,0.386124
Beauty & Grooming,-0.016565,0.193731,1.000000,0.228900,0.224303,0.181270,0.285552,0.338150,0.226007,0.325817,0.135062,0.074352,0.185756,0.410663,0.276384,0.235186,0.151909
Books,-0.030022,0.043866,0.228900,1.000000,-0.034992,0.000291,0.505895,0.107125,0.076643,0.104159,0.018155,0.523970,0.349240,0.341190,0.188083,0.039935,0.046394
Computing,0.061213,0.263430,0.224303,-0.034992,1.000000,0.388337,0.251279,0.460664,0.304751,0.287602,0.367950,0.226793,0.220149,0.054836,0.040121,0.120232,0.206217
Entertainment,0.023333,0.479368,0.181270,0.000291,0.388337,1.000000,0.154749,0.222534,0.105686,0.403240,0.533759,0.093190,0.078790,0.170628,0.238944,0.151672,0.343105
Health & Sports,0.006154,0.163356,0.285552,0.505895,0.251279,0.154749,1.000000,0.509007,0.386683,0.395933,0.030326,0.240040,0.260905,0.246042,0.265342,0.326598,0.115837
Home & Living,0.009600,0.223021,0.338150,0.107125,0.460664,0.222534,0.509007,1.000000,0.923793,0.313366,0.123085,0.209012,0.098967,0.268564,0.127837,0.368711,0.232064
Kids & Baby,0.026787,0.100470,0.226007,0.076643,0.304751,0.105686,0.386683,0.923793,1.000000,0.380593,0.053844,0.257818,0.301744,0.162648,0.098026,0.427298,0.025691
Men's Fashion,-0.021875,0.255954,0.325817,0.104159,0.287602,0.403240,0.395933,0.313366,0.380593,1.000000,0.161870,0.054365,0.138849,0.278682,0.194423,0.339410,0.263474


In [12]:
final_df = move_columns_to_end(final_df,'Mobiles & Tablets','Beauty & Grooming')

In [13]:
final_df.head(2)

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.0,593501.2,999.0,22185.0,2250.0,97804.55,1249.0,495.0,32990.0,94530.0,29158.5,115044.85,1257478.35,34219.3
12,207775.3,0.0,3230.0,53644.0,0.0,130560.0,3018.0,27770.00,0.0,720.0,38498.0,0.0,41868.0,12974.00,30886.00,66022.0


1325 rows (more than 9 not nulls)

In [14]:
final_df_scaled = mse_comparison.scale_df(df=final_df,n_targets=2,scale_method='min_max')
final_df_scaled.head(2)

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.0,593501.2,999.0,22185.0,2250.0,97804.55,1249.0,495.0,32990.0,94530.0,29158.5,115044.85,0.017497,0.027410
12,207775.3,0.0,3230.0,53644.0,0.0,130560.0,3018.0,27770.00,0.0,720.0,38498.0,0.0,41868.0,12974.00,0.000430,0.052884


In [21]:
_, _, _, mse_choose = mse_comparison.mse_comparison(path=None,
                                                            df=final_df_scaled, n_features=14,n_splits=10,
                                                             only_choose_one=True,
                                  random_state=42)

Cell execution time: 2023-04-20 05:34:45.538958


CV progress bar: 0it [00:00, ?it/s]

starting choose one model



Choose one progress bar: 0it [00:00, ?it/s]

Starting all
[0.0028050086329988915, 0.0029814447611815073, 0.002739249862235058, 0.002851009512715355, 0.0028595558268116345, 0.002939350361205158, 0.0029567112093737665, 0.0030670582317592855, 0.003345517892453452, 0.003389842630527864, 0.003390355673249329, 0.0034606721514733934, 0.003488166047657543, 0.003506721182421794, 0.0035122664210613047, 0.003511427848387397, 0.0035164715353928288, 0.0035224904775301583, 0.0035261020722436114, 0.0035280966079892138, 0.00353288475916663, 0.0035400051657585667, 0.0035410907920268367, 0.0035448595518481965, 0.003546188343910117, 0.003547301558959568, 0.003551516115903664, 0.0035518827356047942, 0.0035519011776731823, 0.003551956536422626, 0.0035519950497213265, 0.0035519922653434537, 0.003552058879226784, 0.003552089896891354, 0.003552101779229583, 0.0035521447426258962, 0.0035522228280325625, 0.0035522823105294697, 0.0035523324821523663, 0.0035523364133531014, 0.003552357438129683, 0.00355242649254161, 0.003552479066200304, 0.00355263403214089

...

runtime for ind model:1538.4929180145264 seconds
Finish ind
MSE all: 0.03033981359710171
MSE ind: 0
for ind was chosen


[0.0009879554402926892, 0.00103933814115476, 0.0010254079659963954, 0.0010264662314913265, 0.001051391009362301, 0.0010714027514607176, 0.0010995368474121608, 0.001146921990409123, 0.0019612318365345442, 0.0020725758838909215, 0.002677676607035705, 0.002722731472761382, 0.0027962458393254986, 0.002769247179897274, 0.0025762713510359683, 0.00257643923678731, 0.002576825227349802, 0.0025770742445343935, 0.002577107353817115, 0.0025772492213592133, 0.002577300992680759, 0.002577338809080453, 0.0025773460638990587, 0.0025773871963201953, 0.0025774811687239193, 0.002577495026214994, 0.0025773684587362343, 0.002577319010401442, 0.0025772721395205377, 0.0025773350496708764, 0.0025772907455406152, 0.002577248968161754, 0.0025777821005425333, 0.002577940046611703, 0.0025780100715978027, 0.0025779550291781515, 0.002577945849161516, 0.002577951604820259, 0.002577973813059178, 0.0025779862492996963, 0.002577995212508668, 0.002578003732446709, 0.0025780123498153353, 0.002577994855066915, 0.00257800

CV progress bar: 10it [58:23:05, 21018.60s/it]

runtime for ind model:2907.7391600608826 seconds
runtime for choose one model:21487.01327896118 seconds
MSE for all model:   nan
MSE for ind model:   nan
MSE LOO model:       nan
MSE choose one model:0.00308201847447825
stdev choose one model:0.002188168026972508
Total runtime: 3503.099664405982 minutes



/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:276: RuntimeWarning: Mean of empty slice.
  print(f'MSE for all model:   {np.array(mse_all).mean()}')
/data/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:277: RuntimeWarning: Mean of empty slice.
  print(f'MSE for ind model:   {np.array(mse_ind).mean()}')
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:278: RuntimeWarning: Mean of empty slice.
  print(f'MSE LOO model:       {np.array(mse_loo).mean()}')


In [20]:
mse_all, mse_ind, mse_loo, _ = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_scaled, n_features=14,
                                                                                              max_depth=np.inf,
                        
                              use_pruning_for_loo=True,
                              use_pruning_2_models=True, n_splits=10,n_repeats=1,
                                  random_state=0)

Cell execution time: 2023-04-18 22:33:27.981917


CV progress bar: 0it [00:00, ?it/s]

for all print:
[0.002666940766369099, 0.002928405718788745, 0.0030353920338485416, 0.00391223207655577, 0.004778187157436952, 0.005132578813847604, 0.005173710223597512, 0.005091266997711267, 0.005112518583134361, 0.0051332566847115445, 0.005149099854163251, 0.005156390839462294, 0.0051868298406607645, 0.00519605571077031, 0.005198431985641975, 0.005207895901884616, 0.005214107674952833, 0.005214588119648126, 0.005214348879988399, 0.005218075087207871, 0.005218752240780298, 0.005220321109137576, 0.005221417619205813, 0.005225005734589708, 0.005227220050843102, 0.005231455090775232, 0.005231257253414378, 0.005231281171422454, 0.005231220817629151, 0.005231414009508563, 0.005231807272712197, 0.005231721933819998, 0.00523187353669847, 0.005231900846481325, 0.0052349910623261135, 0.005234920421766401, 0.005234943891895489, 0.005235402850046548, 0.005235501159504717, 0.0052354480950061195, 0.00523533640665854, 0.005235271088856091, 0.005235220071137426, 0.0052352681724227934, 0.005235270517

CV progress bar: 1it [6:00:23, 21623.84s/it]

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:5, value to split:169030.9
   | Impurity of the node: 1.07
   | Impurity of the best split: 1.0
   | Count of observations in node: 1192
   |LOO score: 1.0641167230402815
depth=1   leaf node               | Count of observations in node: 1173
                                  | Prediction of node: [0.002]
                                  | Impurity of the node: 0.07
Leaf id: 2
                                  | Loo score: 0.06
depth=1   leaf node               | Count of observations in node: 19
                                  | Prediction of node: [0.062]
                                  | Impurity of the node: 0.93
Leaf id: 3
                                  | Loo score: 0.996
************************************************************sub tree number 1 **************************************************

...

Root
depth=0| Split rule- x_i to split:3, value to split:470702.375
   | Impurity of the node: 5.06
   | Impurity of the best split: 4.75
   | Count of observations in node: 1193
   |LOO score: 4.840624618543401
************************************************************sub tree number 0 ************************************************************
Root
depth=0   leaf node       | Count of observations in node: 1159
                          | Prediction of node: [0.003]
                          | Impurity of the node: 1.03
Leaf id: 2
************************************************************sub tree number 1 ************************************************************
Root
depth=0   leaf node       | Count of observations in node: 1159
                          | Prediction of node: [0.012]
                          | Impurity of the node: 2.52
Leaf id: 6
************************************************************sub tree number 0 **************************************************

CV progress bar: 9it [28:14:16, 9905.79s/it] 

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:5, value to split:165477.155
   | Impurity of the node: 1.11
   | Impurity of the best split: 1.04
   | Count of observations in node: 1193
   |LOO score: 1.1089242443315837
depth=1   leaf node               | Count of observations in node: 1173
                                  | Prediction of node: [0.002]
                                  | Impurity of the node: 0.11
Leaf id: 2
                                  | Loo score: 0.105
depth=1   leaf node               | Count of observations in node: 20
                                  | Prediction of node: [0.06]
                                  | Impurity of the node: 0.93
Leaf id: 3
                                  | Loo score: 0.992
************************************************************sub tree number 1 ***********************************************

CV progress bar: 10it [31:01:17, 11167.75s/it]

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:3, value to split:1665092.615
   | Impurity of the node: 0.13
   | Impurity of the best split: 0.08
   | Count of observations in node: 1193
   |LOO score: 0.1311327391908295
depth=1   leaf node               | Count of observations in node: 1192
                                  | Prediction of node: [0.002]
                                  | Impurity of the node: 0.08
Leaf id: 2
                                  | Loo score: 0.076
depth=1   leaf node               | Count of observations in node: 1
                                  | Prediction of node: [0.237]
                                  | Impurity of the node: 0.0
Leaf id: 3
************************************************************sub tree number 1 ************************************************************
Root
depth=0| Split rule- x_i to split:

In [22]:
import statistics
print(f'for all stdev {statistics.stdev(mse_all)}')
print(f'for ind stdev {statistics.stdev(mse_ind)}')
print(f'for loo stdev {statistics.stdev(mse_loo)}')

for all stdev 0.0027700617401563085
for ind stdev 0.002688711054970641
for loo stdev 0.0028427957493826045
